**Restricted Boltzmann Machine (RBM)**

In [2]:
import numpy as np

class RBM:
    def __init__(self, visible_units, hidden_units, learning_rate=0.1, epochs=1000):
        self.visible_units = visible_units
        self.hidden_units = hidden_units
        self.learning_rate = learning_rate
        self.epochs = epochs

        # Initialize weights and biases
        self.weights = np.random.normal(0, 0.01, (self.visible_units, self.hidden_units))
        self.hidden_bias = np.zeros((1, self.hidden_units))
        self.visible_bias = np.zeros((1, self.visible_units))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sample(self, probabilities):
        return (np.random.rand(*probabilities.shape) < probabilities).astype(np.float32)

    def train(self, data):
        for epoch in range(self.epochs):
            # Positive phase
            pos_hidden_probs = self.sigmoid(np.dot(data, self.weights) + self.hidden_bias)
            pos_hidden_states = self.sample(pos_hidden_probs)
            pos_associations = np.dot(data.T, pos_hidden_probs)

            # Negative phase
            neg_visible_probs = self.sigmoid(np.dot(pos_hidden_states, self.weights.T) + self.visible_bias)
            neg_visible_states = self.sample(neg_visible_probs)
            neg_hidden_probs = self.sigmoid(np.dot(neg_visible_states, self.weights) + self.hidden_bias)
            neg_associations = np.dot(neg_visible_states.T, neg_hidden_probs)

            # Update weights and biases
            self.weights += self.learning_rate * ((pos_associations - neg_associations) / data.shape[0])
            self.visible_bias += self.learning_rate * np.mean(data - neg_visible_states, axis=0, keepdims=True)
            self.hidden_bias += self.learning_rate * np.mean(pos_hidden_probs - neg_hidden_probs, axis=0, keepdims=True)

            if epoch % 100 == 0:
                error = np.mean((data - neg_visible_states) ** 2)
                print(f"Epoch {epoch}: Reconstruction Error = {error:.4f}")

    def transform(self, data):
        return self.sigmoid(np.dot(data, self.weights) + self.hidden_bias)

    def reconstruct(self, data):
        hidden = self.transform(data)
        return self.sigmoid(np.dot(hidden, self.weights.T) + self.visible_bias)


# Example Usage
if __name__ == "__main__":
    # Generate dummy data: 10 samples, 6 features
    data = np.random.randint(0, 2, (10, 6)).astype(np.float32)

    rbm = RBM(visible_units=6, hidden_units=3, learning_rate=0.1, epochs=1000)
    rbm.train(data)

    print("Original Data:")
    print(data)

    print("Reconstructed Data:")
    print(rbm.reconstruct(data))


Epoch 0: Reconstruction Error = 0.4667
Epoch 100: Reconstruction Error = 0.4667
Epoch 200: Reconstruction Error = 0.4333
Epoch 300: Reconstruction Error = 0.4833
Epoch 400: Reconstruction Error = 0.4500
Epoch 500: Reconstruction Error = 0.5000
Epoch 600: Reconstruction Error = 0.4167
Epoch 700: Reconstruction Error = 0.2833
Epoch 800: Reconstruction Error = 0.4167
Epoch 900: Reconstruction Error = 0.3500
Original Data:
[[1. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 1. 0.]
 [1. 1. 1. 0. 0. 0.]
 [1. 1. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 1. 1. 0. 0. 0.]
 [1. 0. 1. 0. 1. 1.]
 [0. 1. 0. 1. 1. 0.]
 [1. 1. 0. 1. 1. 1.]
 [0. 0. 1. 0. 1. 1.]]
Reconstructed Data:
[[0.64288554 0.58918405 0.57280918 0.21632389 0.51490885 0.39738028]
 [0.6838922  0.58993787 0.65251622 0.14387568 0.48855627 0.39095798]
 [0.69969893 0.59117019 0.68374492 0.11983257 0.47510002 0.38553113]
 [0.63813179 0.58840836 0.56260231 0.22727779 0.51998261 0.40024995]
 [0.52310204 0.58616933 0.34980497 0.51543784 0.58814291 0.41804995]
 